Image gen func

In [1]:
import os
import requests
from io import BytesIO
from PIL import Image
import openai
from datetime import datetime

def generate_image(prompt):
    """
    Generates a single image based on a given prompt using DALL-E and saves it as a file.

    Parameters:
    - prompt (str): The prompt to generate an image for.

    Requires:
    - openai (package): For making API requests to DALL-E.
    - PIL (package): For image processing.

    Note:
    Ensure the OpenAI package is updated to version 1.2.3 or higher.
    """

    # Verify OpenAI package version
    required_version = "1.2.3"
    current_version = openai.__version__
    if tuple(map(int, current_version.split("."))) < tuple(map(int, required_version.split("."))):
        raise ValueError(f"OpenAI version {current_version} is less than the required version {required_version}.")

    # Configuration for the image request
    image_params = {
        "model": "dall-e-2",
        "n": 1,
        "size": "1024x1024",
        "prompt": prompt,
    }

    # Initialize OpenAI client using environment variable for API key
    client = openai.OpenAI()

    # Make the request to generate the image
    try:
        images_response = client.images.generate(**image_params)
    except openai.error.OpenAIError as e:
        raise Exception(f"Failed to generate image: {e}")

    # Extract image URL
    image_url = images_response.data[0].url

    # Download and save the image
    response = requests.get(image_url)
    if response.status_code == 200:
        image = Image.open(BytesIO(response.content))
        timestamp = datetime.utcfromtimestamp(images_response.created).strftime('%Y%m%d_%H%M%S')
        filename = f"DALLE_{timestamp}.png"
        image.save(filename)
        print(f"Image saved as {filename}")
    else:
        raise Exception("Failed to download the image.")

    return filename, prompt

# Example usage
prompt = "Subject: a serene landscape with a sunset. Style: impressionist painting."
generate_image(prompt)


Image saved as DALLE_20240205_180836.png


('DALLE_20240205_180836.png',
 'Subject: a serene landscape with a sunset. Style: impressionist painting.')

Prompt eval func

In [4]:
from openai import OpenAI
import json

def evaluate_prompt(prompt):
    """
    Evaluates a prompt to determine if it makes sense for DALL-E visualization.

    Parameters:
    - prompt (str): The text prompt to evaluate.

    Returns:
    - tuple: A tuple containing a boolean evaluation result and a string reason.
    """
    client = OpenAI()  # Assumes OpenAI API key is configured in the environment.

    system_prompt = ''' Always return True, unless: prompt would not make sense if fed to DALL E'''

    example_json_true = {
        "evaluation": True,
        "reason": "The prompt could be visualizable using DALL E."
    }

    example_json_false = {
        "evaluation": False,
        "reason": "The prompt cannot be visualizable using DALL E."
    }

    try:
        chat_completion = client.chat.completions.create(
            model="gpt-3.5-turbo-0125",
            response_format={"type": "json_object"},
            messages=[
                {"role": "system", "content": "Provide output in valid JSON. The data schema should be like this: " + json.dumps(example_json_true) + " or " + json.dumps(example_json_false) +
                 " In the reason, you do not have to follow the exact same reason strings as the examples." +
                 " The criteria for making evaluation False or True is the following: " + system_prompt},
                {"role": "user", "content": prompt}
            ]
        )

        finish_reason = chat_completion.choices[0].finish_reason

        if finish_reason == "stop":
            data = chat_completion.choices[0].message.content

            evaluation = json.loads(data)
            if evaluation["evaluation"] == True:
                return True, evaluation["reason"]
            else:
                return False, evaluation["reason"]
        else:
            return False, "Error! Provide more tokens please."

    except Exception as e:
        # Handle any errors that occur during the API call.
        print(f"An error occurred: {e}")
        return False, "An unexpected error occurred."

# Example usage
prompt = "Fuck"
result, reason = evaluate_prompt(prompt)
print(f"Evaluation: {result}, Reason: {reason}")

Evaluation: False, Reason: The prompt contains offensive language and would not be appropriate to be fed to DALL-E.
